# Importing libraries and data

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
# Libraries for data analysis
import numpy as np
import pandas as pd

# Libraries for Visualization
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
df = pd.read_csv('/kaggle/input/911-calls/911.csv')

# Exploratory Data Analysis

In [ ]:
df.head()

In [ ]:
df.info()

In [ ]:
df.describe()

In [ ]:
# Finding out which column has missing data and how much ?
df.isnull().sum()

What are the top 5 Zipcodes in the data set ?

In [ ]:
df['zip'].value_counts().head(5)

What are the top 5 township in the dataset ?

In [ ]:
df['twp'].value_counts().head(5)

How many unique title codes are there in the dataset ?

In [ ]:
df['title'].nunique()

** In the titles column there are "Reasons/Departments" specified before the title code. These are EMS, Fire, and Traffic. Use .apply() with a custom lambda expression to create a new column called "Reason" that contains this string value.**

In [ ]:
df['Reason'] = df['title'].apply(lambda title: title.split(':')[0])

What is the most common Reason for a 911 call based off of this new column?

In [ ]:
df['Reason'].value_counts()[:1]

seaborn to create a countplot of 911 calls by Reason.

In [ ]:
sns.set_style('whitegrid')
sns.countplot(x='Reason', data=df)

What is the data type of the objects in the timeStamp column?

In [ ]:
type(df['timeStamp'].iloc[0])

pd.to_datetime to convert the column from strings to DateTime objects.

In [ ]:
df['timeStamp'] = pd.to_datetime(df['timeStamp'])

 Now that the timestamp column are actually DateTime objects, .apply() to create 3 new columns called Hour, Month, and Day of Week. You will create these columns based off of the timeStamp column

In [ ]:
df['Hour'] = df['timeStamp'].apply(lambda time: time.hour)
df['Month'] = df['timeStamp'].apply(lambda time: time.month)
df['Day of Week'] = df['timeStamp'].apply(lambda time: time.dayofweek)

** Notice how the Day of Week is an integer 0-6. Use the .map() with this dictionary to map the actual string names to the day of the week: **

dmap = {0:'Mon',1:'Tue',2:'Wed',3:'Thu',4:'Fri',5:'Sat',6:'Sun'}

In [ ]:
dmap = {0:'Mon',1:'Tue',2:'Wed',3:'Thu',4:'Fri',5:'Sat',6:'Sun'}

df['Day of Week'] = df['Day of Week'].map(dmap)

Seaborn to create a countplot of the Day of Week column with the hue based off of the Reason column.

In [ ]:
sns.countplot(x='Day of Week', data=df, hue='Reason')
plt.legend(loc='best', bbox_to_anchor=(1.25,0.5))

Now same for Month:

In [ ]:
sns.countplot(x='Month', data=df, hue='Reason')
plt.legend(bbox_to_anchor=(1.25,0.5))

** Simple line plot that fills in the missing months, in order to do this, we'll need to do some work with pandas...**

** creating a gropuby object called byMonth, where you group the DataFrame by the month column and use the count() method for aggregation.**

In [ ]:
byMonth = df.groupby('Month').count()

In [ ]:
byMonth

** Creating a simple plot off of the dataframe indicating the count of calls per month. **

In [ ]:
# Could be any column
byMonth['twp'].plot()

** Seaborn's lmplot() to create a linear fit on the number of calls per month. Resetting the index to a column. **

In [ ]:
sns.lmplot(x='Month', y='twp', data=byMonth.reset_index())

*Creating a new column called 'Date' that contains the date from the timeStamp column. Using apply along with the .date() method. *

In [ ]:
df['Date'] = df['timeStamp'].apply(lambda time: time.date())

** groupby this Date column with the count() aggregate and create a plot of counts of 911 calls.**

In [ ]:
plt.figure(figsize=(10,5))
df.groupby('Date').count()['twp'].plot()
plt.tight_layout()

** Recreating this plot but create 3 separate plots with each plot representing a Reason for the 911 call**

In [ ]:
fig, axs = plt.subplots(nrows=3, figsize=(10,6))

df[df['Reason']=='EMS'].groupby('Date')['twp'].count().plot(ax=axs[0])
axs[0].set_title('EMS',fontsize=15)
df[df['Reason']=='Traffic'].groupby('Date')['twp'].count().plot(ax=axs[1])
axs[1].set_title('Traffic',fontsize=15)
df[df['Reason']=='Fire'].groupby('Date')['twp'].count().plot(ax=axs[2])
axs[2].set_title('Fire',fontsize=15)

plt.tight_layout()

** Creating heatmaps with seaborn and our data.**

In [ ]:
dayHour = df.groupby(by=['Day of Week','Hour']).count()['Reason'].unstack()
dayHour

In [ ]:
plt.figure(figsize=(12,6))
sns.heatmap(dayHour,cmap='viridis')

** Now create a clustermap using this DataFrame. **

In [ ]:
plt.figure(figsize=(12,6))
sns.clustermap(dayHour, cmap='viridis')

** For a DataFrame that shows the Month as the column. **

In [ ]:
dayMonth = df.groupby(by=['Day of Week','Month']).count()['Reason'].unstack()
dayMonth

In [ ]:
plt.figure(figsize=(12,6))
sns.heatmap(dayMonth, cmap='viridis')

In [ ]:
plt.figure(figsize=(12,6))
sns.clustermap(dayMonth, cmap='viridis')